# Physiodenoising of Signal

## Step 1: Necessary Libraries

Here I am going to be importing all the necessary Python libraries.

In [ ]:
# #!/usr/bin/python
from __future__ import division

import argparse as ap
import numpy as np
import scipy as sp
import os as os
import nibabel as nib
import json
import string
import random
from subprocess import call
import matplotlib as mpl; mpl.use('TkAgg')
import matplotlib.pyplot as plt
from scipy.signal import argrelmax
%matplotlib notebook

In [ ]:
## Generate random string
def random_string(length):
    return ''.join(random.choice(string.ascii_letters) for m in range(length))

## Step 2: Load data for denoising and get some info from header

In [ ]:
input_dataset = '/Users/cesar/cesar_data/myPython/physio_denoising/BIDS/sub-01brainhack/func/sub-01brainhack_task-rest_run-01_bold.nii.gz'
json_file = "/Users/cesar/cesar_data/myPython/physio_denoising/BIDS/sub-01brainhack/func/sub-01brainhack_task-rest_run-01_bold.json"

input_dataset_dir = os.path.dirname(input_dataset)
data = nib.load(input_dataset)
TR = data.header['pixdim'][4]
nscans = data.header['dim'][4]
nslices = data.header['dim'][3]
print "TR=", TR, "\nNumber of volumes=", nscans, "\nNumber of slices=", nslices

## Step 3: Get slice slice_timings

In [ ]:
json_data = json.load(open(json_file))
slice_timings =  json_data['SliceTiming']
# check correct number of  slice_timings
if nslices != np.size(slice_timings):
    print('Slice timings in {} is not equal to number of slices {}'.format(json_file,nslices))

In [ ]:
print slice_timings

## Step 4: Loading pulse signal

In [ ]:
card_file = "ppg.txt"
pulse = np.genfromtxt(card_file)
card_fs = 10000
card_ts = 1/card_fs

# plot pulse signal
card_time = np.arange(0,card_ts*len(pulse),card_ts)
plt.figure()
plt.plot(card_time,pulse)

## Step 5: Preprocessing of pulse signal

### Decimate the pulse signal to card_fs around 40 Hz to save computational cost

In [ ]:
card_time = np.arange(0,card_ts*len(pulse),card_ts)
fdec_pulse = sp.interpolate.interp1d(card_time,pulse)
card_fs = 40
card_ts = 1. / card_fs
card_newtime = np.arange(0,card_time[-1],card_ts)
pulse_dec = fdec_pulse(card_newtime)
plt.figure()
plt.plot(card_time,pulse)
plt.plot(card_newtime,pulse_dec,'k')
pulse = pulse_dec
card_time = card_newtime

### Remove local frequency trends from pulse signal with savitzky_golay

In [ ]:
window_sg = 20 * card_fs # length of window size in seconds
if window_sg % 2 == 0:
    window_sg = window_sg + 1
order_sg = 3 # order of polynomial used in savitzky_golay filter
pulse_smooth=sp.signal.savgol_filter(pulse,window_sg,order_sg)
pulse_detrend = pulse - pulse_smooth
plt.figure()
plt.plot(card_time,pulse)
plt.plot(card_time,pulse_smooth,'k');
plt.plot(card_time,pulse_detrend,'r')
plt.show()
pulse = pulse_detrend

## Step 7: Identification of cardiac peaks

In [ ]:
card_peak_min_dist = int(round(0.6 / card_ts)) # define minimum separation between peaks in timepoints
print('Minimum time points between two cardiac peaks is {}'.format(card_peak_min_dist))

pulse = np.asarray(pulse)
pulse = pulse - pulse.min()
card_peaks = np.asarray(argrelmax(pulse, order = card_peak_min_dist))
card_peaks = card_peaks.flatten() # convert to numpy.array
ax= plt.figure().gca()
ax.plot(card_time,pulse)
ax.plot(card_time[card_peaks],pulse[card_peaks],linestyle="none",marker='*',markersize=10,color='r')

## Step 8: Doing the same for respiration signal

In [ ]:
resp_file = "resp.txt"
resp = np.genfromtxt(resp_file)
resp_fs = 10000
resp_ts = 1/resp_fs

# plot respiration signal
resp_time = np.arange(0,resp_ts*len(resp),resp_ts)
ax_1 = plt.figure().gca()
ax_1.plot(resp_time,resp)

# decimate the respiratory signal to resp_fs around 40 Hz to save computational cost
resp_time = np.arange(0,resp_ts*len(resp),resp_ts)
fdec_resp = sp.interpolate.interp1d(resp_time,resp)
resp_fs = 40
resp_ts = 1. / resp_fs
resp_newtime = np.arange(0,resp_time[-1],resp_ts)
resp_dec = fdec_resp(resp_newtime)
#plt.figure()
#plt.plot(resp_time,resp,'r');plt.plot(resp_newtime,resp_dec,'k')
resp = resp_dec
resp_time = resp_newtime

# remove local frequency trends from pulse signal with savitzky_golay
window_sg = 40 * resp_fs # length of window size in seconds
if window_sg % 2 == 0:
    window_sg = window_sg + 1
order_sg = 3 # order of polynomial used in savitzky_golay filter
resp_smooth=sp.signal.savgol_filter(resp,window_sg,order_sg)
resp_detrend = resp - resp_smooth
ax_2 = plt.figure().gca()
ax_2.plot(resp_time,resp,'r')
ax_2.plot(resp_time,resp_smooth,'k')
ax_2.plot(resp_time,resp_detrend)

## Step 9: Detecting peaks (maxima) and valleys (minima) in respiration signal

In [ ]:
resp = resp - resp.min()
# define minimum separation between peaks in timepoints
resp_peak_min_dist = int(round(1.5 / resp_ts))
print('Minimum time points between two respiration local maxima (peaks and valleys) is {}'.format(resp_peak_min_dist))
resp_peaks = np.asarray(argrelmax(resp, order = resp_peak_min_dist))
resp_peaks = resp_peaks.flatten()
resp_valleys = np.asarray(argrelmax(-1.0*resp, order = resp_peak_min_dist))
resp_valleys = resp_valleys.flatten()
ax = plt.figure().gca()
ax.plot(resp_time,resp,color='k')
ax.plot(resp_time[resp_peaks],resp[resp_peaks],linestyle="none",marker='*',markersize=10,color='r')
ax.plot(resp_time[resp_valleys],resp[resp_valleys],linestyle="none",marker='*',markersize=10,color='g')

# RETROICOR

## Step 1: Computing phase of cardiac pulse for each slice

### Here is an example for one slice (jj=1)

In [ ]:
jj=1
# initialize varibles for current slice
times_crSlice = TR * np.arange(nscans)+slice_timings[jj]
#print times_crSlice[1:10]
phase_card_crSlice = np.zeros(nscans)
card_peaks_sec = card_peaks / card_fs
for ii in range(nscans):
    previous_card_peaks = np.asarray(np.nonzero(card_peaks_sec < times_crSlice[ii]))
    if np.size(previous_card_peaks) == 0:
        t1 = 0
    else:
        last_peak = previous_card_peaks[0][-1]
        t1 = card_peaks_sec[last_peak]
    next_card_peaks = np.asarray(np.nonzero(card_peaks_sec > times_crSlice[ii]))
    if np.size(next_card_peaks) == 0:
        t2 = nscans * TR
    else:
        next_peak = next_card_peaks[0][0]
        t2 = card_peaks_sec[next_peak]
    phase_card_crSlice[ii] = (2*np.math.pi*(times_crSlice[ii] - t1))/(t2-t1)

### plot cardiac phase for this slice

In [ ]:
ax = plt.figure().gca()
ax.plot(phase_card_crSlice)

## Do it for all slices

In [ ]:
phases_card = np.zeros((nscans,nslices))
card_peaks_sec = card_peaks / card_fs
for jj in range(nslices):
    # generate slice acquisition timings across all scans
    times_crSlice = TR*np.arange(nscans) + slice_timings[jj]
    phase_card_crSlice = np.zeros(nscans)
    for ii in range(nscans):
        previous_card_peaks = np.asarray(np.nonzero(card_peaks_sec < times_crSlice[ii]))
        if np.size(previous_card_peaks) == 0:
            t1 = 0
        else:
            last_peak = previous_card_peaks[0][-1]
            t1 = card_peaks_sec[last_peak]
        next_card_peaks = np.asarray(np.nonzero(card_peaks_sec > times_crSlice[ii]))
        if np.size(next_card_peaks) == 0:
            t2 = nscans * TR
        else:
            next_peak = next_card_peaks[0][0]
            t2 = card_peaks_sec[next_peak]
        phase_card_crSlice[ii] = (2*np.math.pi*(times_crSlice[ii] - t1))/(t2-t1)
    phases_card[:,jj] = phase_card_crSlice

In [ ]:
ax = plt.figure().gca()
ax.plot(phases_card[:,10:12])

## Step 2: Computing phase for respiration signal

### First, compute the histogram of the respiration signal

In [ ]:
plt.figure()
resp_hist, resp_hist_bins, resp_hist_patches = plt.hist(resp,bins=100)

### Here is one example for one slice (jj=1)

In [ ]:
# first compute derivative of respiration signal
resp_diff = np.diff(resp,n=1)

jj=1
# initialize varibles for current slice
times_crSlice = TR * np.arange(nscans)+slice_timings[jj]
#print times_crSlice[1:10]
phase_resp_crSlice = np.zeros(nscans)
resp_peaks_sec = card_peaks / card_fs
for ii in range(nscans):
    iphys = int(max([1,round(times_crSlice[ii] * resp_fs)])) # closest idx in resp waveform
    iphys = min([iphys,len(resp_diff)]) # cannot be longer than resp_diff
    thisBin = np.argmin(abs(resp[iphys] - resp_hist_bins))
    numerator = np.sum(resp_hist[0:thisBin])
    phase_resp_crSlice[ii] = np.math.pi*np.sign(resp_diff[iphys])*(numerator/len(resp))
ax = plt.figure().gca()
ax.plot(phase_card_crSlice)

### Do it for all slices

In [ ]:
phases_resp = np.zeros((nscans,nslices))
for jj in range(nslices):
    # generate slice acquisition timings across all scans
    times_crSlice = TR*np.arange(nscans) + slice_timings[jj]
    phase_resp_crSlice = np.zeros(nscans)
    for ii in range(nscans):
        iphys = int(max([1,round(times_crSlice[ii] * resp_fs)])) # closest idx in resp waveform
        iphys = min([iphys,len(resp_diff)]) # cannot be longer than resp_diff
        thisBin = np.argmin(abs(resp[iphys] - resp_hist_bins))
        numerator = np.sum(resp_hist[0:thisBin])
        phase_resp_crSlice[ii] = np.math.pi*np.sign(resp_diff[iphys])*(numerator/len(resp))
    phases_resp[:,jj] = phase_resp_crSlice
ax = plt.figure().gca()
ax.plot(phases_resp[:,10:15])

## Step 3: Compute RETROICOR regressors

In [ ]:
nharmonics_resp = 2
nharmonics_card = 2
regressors_resp_retroicor = np.zeros((nslices,nscans,2*nharmonics_resp))
regressors_card_retroicor = np.zeros((nslices,nscans,2*nharmonics_card))
for jj in range(nslices):
    # respiration
    phase_resp_crSlice = phases_resp[:,jj]
    for nn in range(nharmonics_resp):
        regressors_resp_retroicor[jj,:,(2*nn)] = np.cos((nn+1)*phase_resp_crSlice)
        regressors_resp_retroicor[jj,:,(2*nn+1)] = np.sin((nn+1)*phase_resp_crSlice)
    # cardiac
    phase_card_crSlice = phases_card[:,jj]
    for nn in range(nharmonics_card):
        regressors_card_retroicor[jj,:,(2*nn)] = np.cos((nn+1)*phase_card_crSlice)
        regressors_card_retroicor[jj,:,(2*nn+1)] = np.sin((nn+1)*phase_card_crSlice)

### plot respiratory regressors for one slice (e.g. jj=4)

In [ ]:
ax_1 = plt.figure().gca()
ax_1.plot(regressors_resp_retroicor[4,:,:])

In [ ]:
freqs = np.fft.fftfreq(nscans, TR)
idx = np.argsort(freqs)
ps_1st_harmonic = np.abs(np.fft.fft(regressors_resp_retroicor[4,:,0]))**2
ps_2nd_harmonic = np.abs(np.fft.fft(regressors_resp_retroicor[4,:,2]))**2
ax_2 = plt.figure().gca()
ax_2.plot(freqs[idx], ps_1st_harmonic[idx],color='k')
ax_2.plot(freqs[idx], ps_2nd_harmonic[idx],color='r')

In [ ]:
ax = plt.figure().gca()
ax.plot(regressors_card_retroicor[4,:,:])

### plot cardiac regressors for one slice (jj=4)

In [ ]:
freqs = np.fft.fftfreq(nscans, TR)
idx = np.argsort(freqs)
ps_1st_harmonic = np.abs(np.fft.fft(regressors_card_retroicor[4,:,0]))**2
ps_2nd_harmonic = np.abs(np.fft.fft(regressors_card_retroicor[4,:,2]))**2
ax_2 = plt.figure().gca()
ax_2.plot(freqs[idx], ps_1st_harmonic[idx],color='k')
ax_2.plot(freqs[idx], ps_2nd_harmonic[idx],color='r')

# RVHR 

## Step 1: Compute respiratory volume signal

In [ ]:
Twin = 6 # Time window to consider for instantaneous variations in respiration
regressors_RV = np.zeros((nscans,nslices))
#ax = plt.figure().gca()
#ax.plot(resp)
for jj in range(nslices):
    rv = np.zeros(nscans)
    times_crSlice = TR * np.arange(nscans)+slice_timings[jj]
    for ii in range(nscans):
        i1 = int(max(0,np.floor((times_crSlice[ii]-0.5*Twin)/resp_ts)))
        i2 = int(min(len(resp),np.floor((times_crSlice[ii]+0.5*Twin)/resp_ts)))
        rv[ii] = np.std(resp[i1:i2])
    rv = rv - np.mean(rv)
    regressors_RV[:,jj] = rv

## Step 2: Convolve with respiratory response function

In [ ]:
time_rrf = np.arange(0,40,TR)
rrf = 0.6*(time_rrf**2.1)*np.exp((-1.0*time_rrf)/1.6) - 0.0023 * (time_rrf**3.54)*np.exp((-1.0*time_rrf)/4.25)
#rrf = rrf / np.max(rrf)
rrf = rrf / (np.sum(rrf**2)**0.5)
regressors_RV_RRF = np.zeros((nscans,nslices))
for jj in range(nslices):
    # convolve respiratory volume with RRF
    regressors_RV_RRF[:,jj] = np.convolve(regressors_RV[:,jj],rrf,mode='same')

In [ ]:
ax_1 = plt.figure().gca()
ax_1.plot(time_rrf,rrf / np.max(rrf))
ax_2 = plt.figure().gca()
ax_2.plot(regressors_RV_RRF[:,4],color='k')
ax_2.plot(regressors_RV[:,4],color='r')

## Step 3: Compute cardiac rate signal

In [ ]:
Twin = 6 # Time window to consider for instantaneous variations in respiration
regressors_CR = np.zeros((nscans,nslices))
num_card_peaks = np.size(card_peaks_sec)
for jj in range(nslices):
    hr = np.zeros(num_card_peaks)
    for ii in range(num_card_peaks):
        if ii == (num_card_peaks - 1):
            hr[ii] = hr[ii-1]
        else:
            hr[ii] = 1.0 / (card_peaks_sec[ii+1]-card_peaks_sec[ii])
    fdec_hr = sp.interpolate.interp1d(card_peaks_sec,hr,fill_value="extrapolate")
    times_crSlice = TR * np.arange(nscans) + slice_timings[jj]
    hr = fdec_hr(times_crSlice)
    #if times_crSlice[0] < card_peaks_sec[0]:
    #    hr = fdec_hr(times_crSlice[1:])
    #    hr = np.append(hr[0],hr)
    #else:
    #    hr = fdec_hr(times_crSlice)
    hr = hr - np.mean(hr)
    regressors_CR[:,jj] = hr

In [ ]:
ax = plt.figure().gca()
ax.plot(regressors_CR[:,1])

## Step 4: Convolve with Cardiac Response Function

In [ ]:
time_crf = np.arange(0,40,TR)
crf = 0.6*(time_crf**2.7)*np.exp((-1.0*time_crf)/1.6) - 16 * sp.stats.norm.pdf(time_crf,12,3)
crf = crf / (np.sum(crf**2)**0.5)
regressors_CR_CRF = np.zeros((nscans,nslices))
for jj in range(nslices):
    # convolve respiratory volume with RRF
    regressors_CR_CRF[:,jj] = np.convolve(regressors_CR[:,jj],crf,mode='same')

In [ ]:
ax_1 = plt.figure().gca()
ax_1.plot(time_crf,crf / np.max(crf))
ax_2 = plt.figure().gca()
ax_2.plot(regressors_CR_CRF[:,4],color='k')
ax_2.plot(regressors_CR[:,4],color='r')

# Denoising data

## Step 1: Split dataset in slices, save regressors in .1D (.txt) files and run 3dDeconvolve for denoising

In [ ]:
crdir = os.getcwd()
input_dataset_dir = os.path.dirname(input_dataset)
print "Directory Input Dataset:", input_dataset_dir
input_fname = os.path.basename(input_dataset)
print "Input Dataset:", input_fname, "\n"
# change to directory of input dataset (this is just for convenience, but not strictly necessary)
os.chdir(input_dataset_dir)

jobs = 6 # number of processors in multi-CPU machines
tmp_file = random_string(15)
for jj in range(nslices):
    
    cr_tmp_file = "%s.SL%02d" % (tmp_file,jj)
    cr_tmp_file_NII = "%s.SL%02d.nii.gz" % (tmp_file,jj)
    command = "3dZcutup -keep %s %s -prefix %s %s" % (jj,jj,cr_tmp_file_NII,input_fname)
    print command, "\n"
    call(command,shell=True)
    
    # Run 3dDeconvolve
    command = "3dDeconvolve -float -quiet -polort A -input %s" % (cr_tmp_file_NII)
    
    # RETROICOR RESPIRATION REGRESSORS
    cr_ortvec_file = "%s_retro_resp.1D" % (cr_tmp_file)
    np.savetxt(cr_ortvec_file,regressors_resp_retroicor[jj,:,:],fmt='%.6f')
    command = "%s -ortvec %s RETRO_RESP" % (command,cr_ortvec_file)
    
    # RETROICOR CARDIAC REGRESSORS
    cr_ortvec_file = "%s_retro_card.1D" % (cr_tmp_file)
    np.savetxt(cr_ortvec_file,regressors_card_retroicor[jj,:,:],fmt='%.6f')
    command = "%s -ortvec %s RETRO_CARD" % (command,cr_ortvec_file)
    
    # RV convolved with RRF REGRESSOR
    cr_ortvec_file = "%s_hr_crf.1D" % (cr_tmp_file)
    np.savetxt(cr_ortvec_file,regressors_CR_CRF[:,jj],fmt='%.6f')
    command = "%s -ortvec %s HR_CRF" % (command,cr_ortvec_file)
    
    # CR convolved with CRF REGRESSOR
    cr_ortvec_file = "%s_rv_rrf.1D" % (cr_tmp_file)
    np.savetxt(cr_ortvec_file,regressors_RV_RRF[:,jj],fmt='%.6f')
    command = "%s -ortvec %s RV_RRF" % (command,cr_ortvec_file)

    # statistics
    command = "%s -fout -rout -bout" % (command)
    
    # denoised dataset
    command = "%s -errts errts_%s" % (command,cr_tmp_file_NII)
    
    # bucket with statistics
    command = "%s -bucket stats_%s" % (command,cr_tmp_file_NII)
    
    # save physiological regressors for current slice
    cr_x1D_file = "X_physio_%s.SL%02d.1D" % (input_fname,jj)
    command = "%s -x1D %s" % (command,cr_x1D_file)
    
    # number of jobs to run 3dDeconvolve on multi-CPU machine
    command = "%s -jobs %d" % (command,jobs)
    
    # run 3dDeconvolve
    print command, "\n"
    call(command,shell=True)
    
    # delete temporary 1D files
    command = "rm %s_*.1D" % (cr_tmp_file)
    print command, "\n"
    call(command,shell=True)
    

## Step 2: Merge denoised dataset

In [ ]:
command = "3dZcat -overwrite -datum float -prefix denoised_%s errts_%s.SL*" % (input_fname,tmp_file)
print command, "\n"
call(command,shell=True)
command = "3dZcat -overwrite -datum float -prefix stats_%s stats_%s.SL*" % (input_fname,tmp_file)
print command, "\n"
call(command,shell=True)

## Step 3: Remove results for each slice and return to original directory

In [ ]:
command = "rm %s.SL* errts_%s.SL* stats_%s.SL*" % (tmp_file,tmp_file,tmp_file)
print command, "\n"
call(command,shell=True)
os.chdir(crdir)

# That's all folks, enjoy denoising and remember to collect physiological signals. Otherwise you won't know whether your data will affected by these confounding signals